# Lab 2, Part 1 - Decision Tree & Random Forest
- Name1, Student's ID1
- Name2, Student's ID2

## Lab Instruction 

In this lab, you are to create a Decision Tree and Random Forest model  to predict the sale price of houses **SalePrice = (Low, Medium, High)** from a given set of attributes. <br>

The data file is`lab2_dataset.csv` <br>

Note that you need to explore and process/drop attributes, <br>
and map numerical values of sale prices to categorical values (Low, Medium, High)

###  1. Import the Dataset and Learn About the Data

In [35]:
import pandas as pd
df = pd.read_csv("./Lab2/lab2_dataset.csv")
print(pd.__version__) # You should use version 0.21+

0.23.4


In [36]:
df.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [37]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 81 columns):
Id               1460 non-null int64
MSSubClass       1460 non-null int64
MSZoning         1460 non-null object
LotFrontage      1201 non-null float64
LotArea          1460 non-null int64
Street           1460 non-null object
Alley            91 non-null object
LotShape         1460 non-null object
LandContour      1460 non-null object
Utilities        1460 non-null object
LotConfig        1460 non-null object
LandSlope        1460 non-null object
Neighborhood     1460 non-null object
Condition1       1460 non-null object
Condition2       1460 non-null object
BldgType         1460 non-null object
HouseStyle       1460 non-null object
OverallQual      1460 non-null int64
OverallCond      1460 non-null int64
YearBuilt        1460 non-null int64
YearRemodAdd     1460 non-null int64
RoofStyle        1460 non-null object
RoofMatl         1460 non-null object
Exterior1st      1460 non-n

###  2. Preprocessing
Try to think about what data's feature that the model accept and how the model compute those data. Then use techniques that you have learned to preprocess the data. 

**For example:** 
-  Remove non-informative features
-  Remove features with too many NA
-  Remove rows with incomplete data
-  Remove features with highly unbalanced labels
-  Encode categorical variables as appropriate

Then, create one dataframe for the features and another frame for the output variable.

In [38]:
null_columns = df.columns[df.isnull().sum() > 500].tolist()

In [39]:
null_columns = null_columns + ['Id']

In [40]:
df2 = df.drop(columns=null_columns)

In [41]:
df2 = df2.dropna()

In [42]:
df2.shape

(1094, 75)

In [43]:
df_num = pd.concat([pd.get_dummies(df2.select_dtypes(include=['object'])),
                    df2.select_dtypes(include=['float64','int64'])],
                   axis=1,
                   sort=True)

In [44]:
df_num.head()

,MSZoning_C (all),MSZoning_FV,MSZoning_RH,MSZoning_RL,MSZoning_RM,Street_Grvl,Street_Pave,LotShape_IR1,LotShape_IR2,LotShape_IR3,...,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SalePrice
0,0,0,0,1,0,0,1,0,0,0,...,0,61,0,0,0,0,0,2,2008,208500
1,0,0,0,1,0,0,1,0,0,0,...,298,0,0,0,0,0,0,5,2007,181500
2,0,0,0,1,0,0,1,1,0,0,...,0,42,0,0,0,0,0,9,2008,223500
3,0,0,0,1,0,0,1,1,0,0,...,0,35,272,0,0,0,0,2,2006,140000
4,0,0,0,1,0,0,1,1,0,0,...,192,84,0,0,0,0,0,12,2008,250000


In [45]:
df_num['salePrice_bin'] = pd.qcut(df_num.SalePrice,q=[0,0.33,0.66,1],labels=['Low','Medium','High'])

In [46]:
df_num.salePrice_bin.value_counts()

High      371
Low       364
Medium    359
Name: salePrice_bin, dtype: int64

In [47]:
y = df_num['salePrice_bin']
X = df_num.drop(columns=['salePrice_bin','SalePrice'])

## Decision Tree Classifier

Use both Hold-out and K-fold CV to evaluate your model

Analyze the model results. Do you think the model is good enough? <br>
Does it overfit or underfit the data? <br>
Explain and provide evidence to support your claims.
Look at various classification matrix of train and test sets.



### 3.1 Hold-out evaluation

In [48]:
# Hold-out Method
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42) 

In [49]:
from sklearn import tree

dt_clf = tree.DecisionTreeClassifier()
dt_clf = dt_clf.fit(X_train,y_train)

In [50]:
dt_clf.score(X_test,y_test)

0.74429223744292239

Evaluate and analyse the result using classification_report module and confusion matrix
- See http://scikit-learn.org/stable/modules/generated/sklearn.metrics.classification_report.html

In [51]:
yhat_train = dt_clf.predict(X_train)
yhat_test = dt_clf.predict(X_test) 
yhat_test_prob = dt_clf.predict_proba(X_test)

In [52]:
from sklearn.metrics import classification_report
print(classification_report(y_train, yhat_train))
print(classification_report(y_test, yhat_test))

             precision    recall  f1-score   support

       High       1.00      1.00      1.00       296
        Low       1.00      1.00      1.00       288
     Medium       1.00      1.00      1.00       291

avg / total       1.00      1.00      1.00       875

             precision    recall  f1-score   support

       High       0.78      0.91      0.84        75
        Low       0.79      0.75      0.77        76
     Medium       0.63      0.56      0.59        68

avg / total       0.74      0.74      0.74       219



In [53]:
# Confusion Matrix
from sklearn.metrics import confusion_matrix
print(confusion_matrix(y_test,yhat_test))

[[68  0  7]
 [ 4 57 15]
 [15 15 38]]


### Visualize  Tree 
- We will use https://github.com/xflr6/graphviz

To install this package, type the following command in the Anaconda Prompt.

```conda install -c conda-forge graphviz python-graphviz ```

Run the following cell to visualize your graph

In [54]:
import graphviz
dot_data = tree.export_graphviz(dt_clf, out_file=None,
                                feature_names=X_train.columns,
                                class_names=['Low','Medium','High'], 
                                filled=True, 
                                rounded=True, 
                                special_characters=True, 
                                proportion=True)
graph = graphviz.Source(dot_data) 
graph.render("tree")

'tree.pdf'

The output from the code above is a pdf file that have a visualization of your decision tree model. Open the pdf file and analyse your model

### 3.2 K-Fold CV 


In [55]:
from sklearn.model_selection import cross_val_score

score = cross_val_score(tree.DecisionTreeClassifier(), X, y, cv = 5)
print(score)
print("5-Fold Cross Validation Accuracy : %1.4f" % score.mean())

[ 0.7         0.71689498  0.78082192  0.73972603  0.76036866]
5-Fold Cross Validation Accuracy : 0.7396


# <font color='red'> Answer:</font>



The model appears to overfit the data as the train performance is substantially better than the test performance.

## Random Forest Classifier

Use both Hold-out and K-fold CV to evaluate the classifier. 

Analyze the model results. 
- Do you think the model is good enough? Does it overfit or underfit the data? 
- How does random forest classifier perform compared to a basic decision tree classifier ? <br>

Explain and provide evidence to support your claims.

### 4.1 Hold-out evaluation

In [58]:
from sklearn.ensemble import RandomForestClassifier

rf_clf = RandomForestClassifier()
rf_clf = rf_clf.fit(X_train, y_train)

In [59]:
rf_clf.score(X_test,y_test)

0.79908675799086759

Evaluate and analyse your result using classification_report module and confusion matrix
- See http://scikit-learn.org/stable/modules/generated/sklearn.metrics.classification_report.html

In [62]:
yhat_train = rf_clf.predict(X_train)
yhat_test = rf_clf.predict(X_test) 
yhat_test_prob = rf_clf.predict_proba(X_test)

In [63]:
from sklearn.metrics import classification_report
print(classification_report(y_train, yhat_train))
print(classification_report(y_test, yhat_test))

             precision    recall  f1-score   support

       High       0.99      1.00      1.00       296
        Low       0.99      1.00      0.99       288
     Medium       1.00      0.99      0.99       291

avg / total       0.99      0.99      0.99       875

             precision    recall  f1-score   support

       High       0.87      0.91      0.89        75
        Low       0.80      0.86      0.83        76
     Medium       0.70      0.62      0.66        68

avg / total       0.79      0.80      0.80       219



### 4.2 K-Fold CV with accuracy metric


In [64]:
from sklearn.model_selection import cross_val_score

score = cross_val_score(RandomForestClassifier(), X, y, cv = 5)
print(score)
print("5-Fold Cross Validation Accuracy : %1.4f" % score.mean())

[ 0.77727273  0.78538813  0.78995434  0.82191781  0.78801843]
5-Fold Cross Validation Accuracy : 0.7925


### 4.3 Evaluate using multiple metrics
- precision: http://scikit-learn.org/stable/modules/generated/sklearn.metrics.precision_score.html
- recall: http://scikit-learn.org/stable/modules/generated/sklearn.metrics.recall_score.html

In [66]:
from sklearn.model_selection import cross_validate

scoring = ['accuracy','precision_macro', 'recall_macro']
scores = cross_validate(rf_clf, X, y, scoring=scoring, cv = 5, return_train_score= True)
pd.DataFrame(scores)

,fit_time,score_time,test_accuracy,train_accuracy,test_precision_macro,train_precision_macro,test_recall_macro,train_recall_macro
0,0.026505,0.006970,0.786364,0.991991,0.783603,0.992080,0.784777,0.991921
1,0.024537,0.007554,0.808219,0.988571,0.811081,0.988726,0.806435,0.988402
2,0.021962,0.006687,0.789954,0.989714,0.790731,0.989857,0.789174,0.989586
3,0.021225,0.005988,0.776256,0.993143,0.777614,0.993251,0.775596,0.993031
4,0.022868,0.005975,0.815668,0.995439,0.815414,0.995503,0.814452,0.995370


# <font color='red'> Answer:</font>

It seems that RF still over fits the data but it is somewhat better than Decision Tree.

### 4.4 Parameter Tuning using GridSearch

Try grid search on parameters max_depth, max_features, and n_estimators.
Determine which parameter set  achieve the best result. <br>


- See more: http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html

In [74]:
from sklearn.model_selection import GridSearchCV

param_grid = {'max_depth':[5, 10, 20], 'max_features':[5, 15, 25], 'n_estimators':[10,25,50,75]}
rf_clf = GridSearchCV(RandomForestClassifier(), param_grid)
rf_clf.fit(X_train, y_train)

GridSearchCV(cv=None, error_score='raise',
       estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'max_depth': [5, 10, 20], 'max_features': [5, 15, 25], 'n_estimators': [10, 25, 50, 75]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [68]:
import sklearn as sk
print(sk.__version__)

0.19.1


In [72]:
s = pd.DataFrame(rf_clf.cv_results_)
s.sort_values('mean_test_score',ascending=False)

/Users/peerapon/anaconda3/lib/python3.6/site-packages/sklearn/utils/deprecation.py:122: FutureWarning: You are accessing a training score ('split0_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
/Users/peerapon/anaconda3/lib/python3.6/site-packages/sklearn/utils/deprecation.py:122: FutureWarning: You are accessing a training score ('split1_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
/Users/peerapon/anaconda3/lib/python3.6/site-packages/sklearn/utils/deprecation.py:122: FutureWarning: You are accessing a training score ('split2_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
/Users/peerapon/anacon

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_max_depth,param_max_features,param_n_estimators,params,split0_test_score,split1_test_score,split2_test_score,mean_test_score,std_test_score,rank_test_score,split0_train_score,split1_train_score,split2_train_score,mean_train_score,std_train_score
14,0.073141,0.001597,0.005558,0.001167,10,15,50,"{'max_depth': 10, 'max_features': 15, 'n_estim...",0.784247,0.825342,0.869416,0.826286,0.034767,1,0.998285,0.994854,0.991438,0.994859,0.002795
26,0.082646,0.005624,0.004864,0.000315,20,25,50,"{'max_depth': 20, 'max_features': 25, 'n_estim...",0.811644,0.821918,0.841924,0.825143,0.012567,2,1.000000,1.000000,1.000000,1.000000,0.000000
23,0.067846,0.005774,0.004899,0.000566,20,15,50,"{'max_depth': 20, 'max_features': 15, 'n_estim...",0.784247,0.815068,0.859107,0.819429,0.030708,3,1.000000,1.000000,1.000000,1.000000,0.000000
20,0.056248,0.002979,0.004810,0.000200,20,5,50,"{'max_depth': 20, 'max_features': 5, 'n_estima...",0.808219,0.794521,0.845361,0.816000,0.021468,4,1.000000,1.000000,1.000000,1.000000,0.000000
8,0.066655,0.003959,0.004440,0.000404,5,25,50,"{'max_depth': 5, 'max_features': 25, 'n_estima...",0.797945,0.804795,0.838488,0.813714,0.017710,5,0.926244,0.917667,0.900685,0.914865,0.010621
17,0.074704,0.001585,0.004796,0.000501,10,25,50,"{'max_depth': 10, 'max_features': 25, 'n_estim...",0.784247,0.801370,0.848797,0.811429,0.027290,6,0.998285,0.998285,0.993151,0.996573,0.002420
16,0.029326,0.000220,0.002307,0.000069,10,25,20,"{'max_depth': 10, 'max_features': 25, 'n_estim...",0.808219,0.797945,0.824742,0.810286,0.011034,7,0.996569,0.998285,0.994863,0.996572,0.001397
7,0.029711,0.000176,0.002766,0.000410,5,25,20,"{'max_depth': 5, 'max_features': 25, 'n_estima...",0.794521,0.804795,0.831615,0.810286,0.015630,7,0.914237,0.900515,0.905822,0.906858,0.005650
25,0.031454,0.000603,0.002403,0.000083,20,25,20,"{'max_depth': 20, 'max_features': 25, 'n_estim...",0.794521,0.808219,0.821306,0.808000,0.010933,9,0.996569,1.000000,0.993151,0.996573,0.002796
15,0.016926,0.000705,0.001675,0.000095,10,25,10,"{'max_depth': 10, 'max_features': 25, 'n_estim...",0.801370,0.821918,0.800687,0.808000,0.009854,9,0.981132,0.994854,0.982877,0.986288,0.006099


Just ignore the warning messages because we are still running version 0.19.1 

**Print out the result for each parameter set**

http://scikit-learn.org/stable/auto_examples/model_selection/plot_grid_search_digits.html

In [75]:
means = rf_clf.cv_results_['mean_test_score']
stds = rf_clf.cv_results_['std_test_score']
for mean, std, params in zip(means, stds, rf_clf.cv_results_['params']):
    print("%0.3f (+/-%0.03f) for %r"
              % (mean, std * 2, params))
print()
print(rf_clf.best_params_, rf_clf.best_score_)


0.747 (+/-0.035) for {'max_depth': 5, 'max_features': 5, 'n_estimators': 10}
0.769 (+/-0.006) for {'max_depth': 5, 'max_features': 5, 'n_estimators': 25}
0.760 (+/-0.059) for {'max_depth': 5, 'max_features': 5, 'n_estimators': 50}
0.753 (+/-0.011) for {'max_depth': 5, 'max_features': 5, 'n_estimators': 75}
0.774 (+/-0.027) for {'max_depth': 5, 'max_features': 15, 'n_estimators': 10}
0.799 (+/-0.033) for {'max_depth': 5, 'max_features': 15, 'n_estimators': 25}
0.785 (+/-0.019) for {'max_depth': 5, 'max_features': 15, 'n_estimators': 50}
0.797 (+/-0.014) for {'max_depth': 5, 'max_features': 15, 'n_estimators': 75}
0.793 (+/-0.023) for {'max_depth': 5, 'max_features': 25, 'n_estimators': 10}
0.803 (+/-0.053) for {'max_depth': 5, 'max_features': 25, 'n_estimators': 25}
0.809 (+/-0.042) for {'max_depth': 5, 'max_features': 25, 'n_estimators': 50}
0.811 (+/-0.036) for {'max_depth': 5, 'max_features': 25, 'n_estimators': 75}
0.753 (+/-0.025) for {'max_depth': 10, 'max_features': 5, 'n_estimat

# <font color='red'> Answer:</font>

The best parameter set and mean score are {'max_depth': 10, 'max_features': 15, 'n_estimators': 50} 0.837